In [1]:
import sys
import os

# one level up from the Notebooks directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

# Import environment setup
import setup

# Now you can safely import your modules
from data_pipeline.pipeline import EnergyDemandForecasting
import pandas as pd


In [2]:
from config.config import FEATURE_ENGINEERED_DATA_PATH, INFERENCE_DATA_PATH, RAW_DATA_PATH

### For Day-ahead


In [3]:
ed = EnergyDemandForecasting()

#data = ed.featured_data_loading(FEATURE_ENGINEERED_DATA_PATH=FEATURE_ENGINEERED_DATA_PATH)
#data.head(2)

In [4]:
inf_data = pd.read_csv(RAW_DATA_PATH)
inf_data.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01 00:00:00,232.483571,5.124673,1.191402,0.399029,0.907766,46.614845,26.399566,0.0,1.055937,0,0,17.842767,64.523353,0.416156,1.233158


In [5]:
inf_data.tail(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
49999,2025-06-04 19:45:00,237.579055,17.904864,4.253821,1.050729,0.939617,7.220105,24.808317,0.0,-1.584342,0,0,15.083737,43.051294,0.360774,4.361657


In [6]:
inf_data['Timestamp'] = pd.to_datetime(inf_data['Timestamp'])  # ensure it's datetime
inf_data = inf_data.set_index('Timestamp') 
inf_data = inf_data[inf_data.columns].resample('D').mean()

inf_data.tail(3)

,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
Timestamp,,,,,,,,,,,,,,,
2025-06-02,229.421422,26.679700,6.122319,1.566531,0.900706,24.722421,16.057297,0.068809,-0.240128,0.104167,0.031250,24.733840,47.746748,0.301807,6.091351
2025-06-03,230.035298,27.236029,6.261330,1.580401,0.903957,24.137611,15.589334,0.007507,0.191340,0.156250,0.010417,24.754585,47.403113,0.320828,6.266547
2025-06-04,230.726101,28.221753,6.512869,1.602481,0.901393,26.763867,14.991984,0.023167,-0.110050,0.125000,0.012500,24.124967,49.540634,0.308253,6.528751


In [7]:
inf_data = inf_data.reset_index(drop=False)
inf_data.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01,229.442065,23.274581,5.33817,1.335934,0.904576,24.128362,15.61487,0.04442,-0.005502,0.072917,0.020833,24.458245,49.931362,0.307196,5.341409


### forecast for 10 days
### each data point t, train until t
### forecast t+1
### plot forecast value, uncertainity bounds, actual values

In [8]:
n = len(inf_data)
inf_training_data = inf_data[:n-1]
inf_training_data.tail(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
519,2025-06-03,230.035298,27.236029,6.26133,1.580401,0.903957,24.137611,15.589334,0.007507,0.19134,0.15625,0.010417,24.754585,47.403113,0.320828,6.266547


In [9]:
inf_set = inf_data[n-10:]
inf_set.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
511,2025-05-26,230.123679,26.533685,6.112369,1.552541,0.898765,25.205411,14.641372,0.048538,0.193632,0.104167,0.041667,24.879234,50.752091,0.298928,6.086054


In [10]:
inf_training_data = ed.preprocessing(inf_training_data, inference=False)
inf_set = ed.preprocessing(inf_set, inference=True)

/Users/pranavsrinivasvenkatesh/Projects/Data Science Projects/Electricity Demand Forecasting/data_pipeline/pipeline.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['load_da'] = df['Predicted Load (kW)'].shift(-1)
/Users/pranavsrinivasvenkatesh/Projects/Data Science Projects/Electricity Demand Forecasting/data_pipeline/pipeline.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['load_d2'] = df['Predicted Load (kW)'].shift(-2)
/Users/pranavsrinivasvenkatesh/Projects/Data Science Projects/Electric

In [11]:
inf_training_data.head(1)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),...,Electricity Price (USD/kWh),load_da,load_d2,lag_1,lag_2,lag_3,pct_tmp_chg,pow_cons_chg,roll_mean_load3,roll_std_load3
3,2024-01-04,230.450687,28.109668,6.483036,1.580661,0.905492,24.628754,16.36928,0.028228,-0.227224,...,0.319243,6.122499,5.881918,6.096116,6.339775,5.341409,0.029197,0.060463,6.310604,0.201493


In [12]:
#inf_training_data.isna().sum()
#inf_set.head(10)
inf_set = inf_set.iloc[[-1]]
inf_set

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Temperature (°C),Humidity (%),Electricity Price (USD/kWh),lag_1,lag_2,lag_3,pct_tmp_chg,pow_cons_chg,roll_mean_load3,roll_std_load3
520,2025-06-04,230.726101,28.221753,6.512869,1.602481,0.901393,26.763867,14.991984,0.023167,-0.11005,24.124967,49.540634,0.308253,6.266547,6.091351,6.289963,-0.025434,0.040173,6.295549,0.220138


In [13]:
inf_train_X, inf_train_y = ed.get_model_full_training_datasets(inf_training_data)


In [14]:
inf_set.columns

Index(['Timestamp', 'Voltage (V)', 'Current (A)', 'Power Consumption (kW)',
       'Reactive Power (kVAR)', 'Power Factor', 'Solar Power (kW)',
       'Wind Power (kW)', 'Grid Supply (kW)', 'Voltage Fluctuation (%)',
       'Temperature (°C)', 'Humidity (%)', 'Electricity Price (USD/kWh)',
       'lag_1', 'lag_2', 'lag_3', 'pct_tmp_chg', 'pow_cons_chg',
       'roll_mean_load3', 'roll_std_load3'],
      dtype='object')

In [15]:
inf_set.set_index('Timestamp', inplace=True)
inf_train_X.set_index('Timestamp', inplace=True)

In [16]:
print("Inf traing Data: ", inf_train_X.shape, inf_train_y.shape)
print("Inf set Data: ", inf_set.shape)

Inf traing Data:  (515, 19) (515,)
Inf set Data:  (1, 19)


### Model Training

In [17]:
#rf
rf_mod = ed.model_training(X_train=inf_train_X, y_train=inf_train_y, model_name='rf')

In [18]:
xgb_mod = ed.model_training(X_train=inf_train_X, y_train=inf_train_y, model_name='xgb')

Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fits
Fitting 6 folds for each of 1 candidates, totalling 6 fi

In [19]:
rf_train_pred = ed.model_predict(Xtest=inf_train_X, model=rf_mod)
xgb_train_pred = ed.model_predict(Xtest=inf_train_X, model=xgb_mod)

In [20]:
import numpy as np
meta_X_train = np.column_stack((rf_train_pred, xgb_train_pred))
meta_mod = ed.model_training(X_train=meta_X_train, y_train=inf_train_y, model_name='meta')

Fitting 6 folds for each of 108 candidates, totalling 648 fits


/Users/pranavsrinivasvenkatesh/anaconda3/envs/streamlitenv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/pranavsrinivasvenkatesh/anaconda3/envs/streamlitenv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/pranavsrinivasvenkatesh/anaconda3/envs/streamlitenv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/pranavsrinivasvenkatesh/anaconda3/envs/streamlitenv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stoc

In [21]:
qrf_model = ed.model_training(X_train=inf_train_X, y_train=inf_train_y, model_name='qrf')

Fitting 6 folds for each of 1 candidates, totalling 6 fits
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_estimators=668, subsample=0.7070593162427692; total time=   0.4s
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_estimators=668, subsample=0.7070593162427692; total time=   0.7s
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_estimators=668, subsample=0.7070593162427692; total time=   1.0s
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_estimators=668, subsample=0.7070593162427692; total time=   1.3s
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_estimators=668, subsample=0.7070593162427692; total time=   1.5s
[CV] END learning_rate=0.0016994636371262762, max_depth=6, min_samples_leaf=8, min_samples_split=7, n_esti

In [23]:
y_pred_rf = rf_mod.predict(inf_set)
y_pred_xgb = xgb_mod.predict(inf_set)
meta_inf = np.column_stack((y_pred_rf, y_pred_xgb))
y_pred_meta = meta_mod.predict(meta_inf)

In [24]:
preds = {}
for q, mod in qrf_model.items():
    preds[q] = mod.predict(inf_set)

In [25]:
preds

{0.1: array([5.95250737]), 0.5: array([6.31663677]), 0.9: array([6.67644182])}

In [26]:
print("Meta:", y_pred_meta)

Meta: [6.31653495]


In [27]:
print("Actual: 4.361657")

Actual: 4.361657
